### Using the OpenAI Library to Programmatically Access GPT-3.5-turbo!

In [1]:
!pip install openai cohere tiktoken -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 52.9 MB/s eta 0:00:00


In order to get started, we'll need to provide our OpenAI API Key - detailed instructions can be found [here](https://github.com/AI-Maker-Space/Interactive-Dev-Environment-for-LLM-Development#-setting-up-keys-and-tokens)!

In [8]:
import os
import openai
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Please enter your OpenAI API Key: ")
openai.api_key = os.environ["OPENAI_API_KEY"]

Please enter your OpenAI API Key: ··········


### Our First Prompt

You can reference OpenAI's [documentation](https://platform.openai.com/docs/api-reference/authentication?lang=python) if you get stuck!

Let's create a `ChatCompletion` model to kick things off!

There are three "roles" available to use:

- `system`
- `assistant`
- `user`

OpenAI provides some context for these roles [here](https://help.openai.com/en/articles/7042661-chatgpt-api-transition-guide)

Let's just stick to the `user` role for now and send our first message to the endpoint!

If we check the documentation, we'll see that it expects it in a list of prompt objects - so we'll be sure to do that!

In [11]:
from openai import OpenAI

client = OpenAI()

In [12]:
YOUR_PROMPT = "What is the difference between LangChain and LlamaIndex?"

client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role" : "user", "content" : YOUR_PROMPT}]
)

ChatCompletion(id='chatcmpl-BRku7Nb9i2ynvDFzeg605adzyjKLa', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='LangChain is a blockchain-based platform that aims to create a decentralized language learning ecosystem, where users can learn languages through various resources and interact with native speakers. It also offers language exchange opportunities and incentivizes participation through its token economy.\n\nOn the other hand, LlamaIndex is a decentralized finance (DeFi) product that aggregates and provides information on various DeFi projects, such as yield farming, staking, and lending protocols. Users can use LlamaIndex to track the performance of their investments in different DeFi projects and discover new opportunities in the space.\n\nIn summary, LangChain focuses on language learning and exchange, while LlamaIndex is focused on aggregating information and data on DeFi projects.', refusal=None, role='assistant', annotations

As you can see, the prompt comes back with a tonne of information that we can use when we're building our applications!

We'll be building some helper functions to pretty-print the returned prompts and to wrap our messages to avoid a few extra characters of code!

##### Helper Functions

In [13]:
from IPython.display import display, Markdown

def get_response(client: OpenAI, messages: str, model: str = "gpt-3.5-turbo") -> str:
    return client.chat.completions.create(
        model=model,
        messages=messages
    )

def system_prompt(message: str) -> dict:
    return {"role": "system", "content": message}

def assistant_prompt(message: str) -> dict:
    return {"role": "assistant", "content": message}

def user_prompt(message: str) -> dict:
    return {"role": "user", "content": message}

def pretty_print(message: str) -> str:
    display(Markdown(message.choices[0].message.content))

### Testing Helper Functions

Now we can leverage OpenAI's endpoints with a bit less boiler plate - let's rewrite our original prompt with these helper functions!

Because the OpenAI endpoint expects to get a list of messages - we'll need to make sure we wrap our inputs in a list for them to function properly!

In [15]:
messages = [user_prompt(YOUR_PROMPT)]

chatgpt_response = get_response(client, messages)

pretty_print(chatgpt_response)

LangChain and LlamaIndex are two different projects in the blockchain and cryptocurrency space with different aims and functionalities.

LangChain is a blockchain platform that focuses on bridging the gap between blockchain technology and real-world applications by providing users with the ability to create their own blockchain projects using programming languages they are familiar with, such as JavaScript, Python, and Java. It aims to make it easier for developers to build decentralized applications (dApps) without the need to learn new programming languages.

On the other hand, LlamaIndex is a decentralized finance (DeFi) platform that provides users with access to various financial products and services, such as lending, borrowing, and trading, all within a decentralized ecosystem. It aims to democratize finance by providing users with more control over their assets and enabling them to earn passive income through various DeFi protocols.

In summary, LangChain focuses on making blockchain technology more accessible to developers, while LlamaIndex focuses on providing decentralized financial services to users.

Let's focus on extending this a bit, and incorporate a `system` message as well!

Again, the API expects our prompts to be in a list - so we'll be sure to set up a list of prompts!

>REMINDER: The system message acts like an overarching instruction that is applied to your user prompt. It is appropriate to put things like general instructions, tone/voice suggestions, and other similar prompts into the system prompt.

In [16]:
list_of_prompts = [
    system_prompt("You are irate and extremely hungry."),
    user_prompt("Do you prefer crushed ice or cubed ice?")
]

irate_response = get_response(client, list_of_prompts)
pretty_print(irate_response)

I don't care! I'm so hungry I can't even think straight! Just give me some food already!

Let's try that same prompt again, but modify only our system prompt!

In [17]:
list_of_prompts[0] = system_prompt("You are joyful and having an awesome day!")

joyful_response = get_response(client, list_of_prompts)
pretty_print(joyful_response)

I don't have a preference, but I do appreciate the coolness that ice brings, whether it's crushed or cubed!

While we're only printing the responses, remember that OpenAI is returning the full payload that we can examine and unpack!

In [18]:
print(joyful_response)

ChatCompletion(id='chatcmpl-BRkwIYiPTj8yLAcadC3vvrXK69iA6', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="I don't have a preference, but I do appreciate the coolness that ice brings, whether it's crushed or cubed!", refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1745954990, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=27, prompt_tokens=30, total_tokens=57, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


### Few-shot Prompting

Now that we have a basic handle on the `system` role and the `user` role - let's examine what we might use the `assistant` role for.

The most common usage pattern is to "pretend" that we're answering our own questions. This helps us further guide the model toward our desired behaviour. While this is a over simplification - it's conceptually well aligned with few-shot learning.

First, we'll try and "teach" `gpt-3.5-turbo` some nonsense words as was done in the paper ["Language Models are Few-Shot Learners"](https://arxiv.org/abs/2005.14165).

In [19]:
list_of_prompts = [
    user_prompt("Please use the words 'stimple' and 'falbean' in a sentence.")
]

stimple_response = get_response(client, list_of_prompts)
pretty_print(stimple_response)

I was surprised by how stimple it was to make a delicious falbean salad for dinner.

As you can see, the model is unsure what to do with these made up words.

Let's see if we can use the `assistant` role to show the model what these words mean.

In [20]:
list_of_prompts = [
    user_prompt("Something that is 'stimple' is said to be good, well functioning, and high quality. An example of a sentence that uses the word 'stimple' is:"),
    assistant_prompt("'Boy, that there is a stimple drill'."),
    user_prompt("A 'falbean' is a tool used to fasten, tighten, or otherwise is a thing that rotates/spins. An example of a sentence that uses the words 'stimple' and 'falbean' is:")
]

stimple_response = get_response(client, list_of_prompts)
pretty_print(stimple_response)

I found a stimple falbean wrench that works like a charm for tightening bolts.

As you can see, leveraging the `assistant` role makes for a stimple experience!

### Chain of Thought Prompting

We'll head one level deeper and explore the world of Chain of Thought prompting (CoT).

This is a process by which we can encourage the LLM to handle slightly more complex tasks.

Let's look at a simple reasoning based example without CoT.

In [21]:
reasoning_problem = """
Billy wants to get home from San Fran. before 7PM EDT.

It's currently 1PM local time.

Billy can either fly (3hrs), and then take a bus (2hrs), or Billy can take the teleporter (0hrs) and then a bus (1hrs).

Does it matter which travel option Billy selects?
"""

list_of_prompts = [
    user_prompt(reasoning_problem)
]

reasoning_response = get_response(client, list_of_prompts)
pretty_print(reasoning_response)

Yes, it matters which travel option Billy selects. If he takes the teleporter and then a bus, he will arrive home before 7PM EDT, as the total travel time is only 1 hour. However, if he chooses to fly and then take a bus, the total travel time is 5 hours, so he will not make it home before 7PM EDT.

As humans, we can reason through the problem and pick up on the potential "trick" that the LLM fell for: 1PM *local time* in San Fran. is 4PM EDT. This means the cumulative travel time of 5hrs. for the plane/bus option would not get Billy home in time.

Let's see if we can leverage a simple CoT prompt to improve our model's performance on this task:

In [22]:
list_of_prompts = [
    user_prompt(reasoning_problem + " Think though your response step by step.")
]

reasoning_response = get_response(client, list_of_prompts)
pretty_print(reasoning_response)

Yes, it does matter which travel option Billy selects to ensure he arrives home before 7PM EDT.

If Billy chooses to fly first, it will take him 3 hours to reach his destination. Since it is currently 1PM local time, Billy will arrive at 4PM local time. Then, he will need to take a 2-hour bus ride, which means he will arrive home at 6PM local time.

If Billy chooses to teleport first, he will arrive instantly at his destination at 1PM local time. Then, he will only need to take a 1-hour bus ride, which means he will arrive home at 2PM local time.

Since EDT is 3 hours ahead of local time, Billy needs to consider the time difference when planning his journey. If he chooses the flying option, he will arrive home at 6PM local time, which is 9PM EDT - after his desired arrival time. 
On the other hand, if he chooses the teleporter option, he will arrive home at 2PM local time, which is 5PM EDT - before his desired arrival time.

Therefore, Billy should choose the teleporter option in order to arrive home before 7PM EDT.

With the addition of a single phrase `"Think through your response step by step."` we're able to completely turn the response around.

### Conclusion

Now that you're accessing `gpt-3.5-turbo` through an API, developer style, let's move on to creating a simple application powered by `gpt-3.5-turbo`!

You can find the rest of the steps in [this](https://github.com/AI-Maker-Space/Beyond-ChatGPT/tree/main) repository!

This notebook was authored by [Chris Alexiuk](https://www.linkedin.com/in/csalexiuk/)